## Assigment 1. Part b. Implemention GloVE

*Done by Polina Turishhcheva, Fall 2020*

### Summary from GloVE Algorythm :
 * Based on the text corpus build a matrix X, which would show how often a word x appear with word y such that there no more than n-2 words between them, where n is size of the context window.
 * Than we randomly initiate a matrix W of size (2*vocab_size, vector_size), which is doubled because each word has a vector for it in a role of main word and in the role of a context word. We also randomly initiate matrixes for biases and gradients. Than we start the training, which is iterating over the pairs of words and changing the values in all randmoly initiaited matrixes according to AdaGrad methodology.
 The cost function from the article is   
 $\sum_{i,j=1}^{vocab}f_{i,j}*(w_i^T w_j~ + b_i +b_j - log(f_{i,j}))^2$  
 where $f_{i,j}$ is score from coocurennce matrix
 AdaGrad updates are   
 score from coocurennce matrix and $b_i , b_j$  are constant, with respenct to $w_i, w_j$, hence, when we derivate we are going to derivate a complex function, the $f_{i,j}$ constant will stay, the corresponding multiplier will go out and the power will decrease. We will get something like
 $J'_{w_i} = \sum_{j=1}^{vocab} f_{i,j}*w_j*(w_i^T w_j~ + b_i +b_j - log(f_{i,j}))$
 and AdaGrad makes updates as $x (t+1, i) = x(t,i)-\frac{mu*g_(t,i)}{\sum_{t'=1}^{t-1} g^2_{t',i}}$



**Choice of hyperparameters** 
* Based on https://www.aclweb.org/anthology/D14-1162.pdf :
  * context window size = 10
  * α = 3/4 ("We [empirically] found that α = 3/4 gives a mod- est improvement over a linear version with α = 1")
  * 50 iterations ("We run 50 iterations for vectors smaller than 300 dimensions, and 100 iterations otherwise")
  * learning_rate = 0.05 and AdaGrad optimizer ("train the model using AdaGrad (Duchi et al., 2011), stochastically sampling non- zero elements from X, with initial learning rate of 0.05.")

  I have used the output vector size of length 100 for because colab provides limited amount of memory

**Disclaimer: this code takes a lot to work**
* I have tried to put training to GPU but it appeared to be even longer (around 90 min per iteration instead of 9 minutes). Eeverything was put to GPU memory and most of the operations were interchanged with PyTorch build-in operations but it did not helped still. Maybe using dataloader can help but anyway, I do not expect it to work 10 times faster after including DataLoader, hence, using CPU appeared to be better in this case.
* Colab has just one core, hence, speeding up via multiprocessing would not work
* Sometimes it crashes on the stage of training for Gutenberg dataset due to lack of RAM but reruning from the begining usually helps

In [1]:
!gdown --id 1eMndxf7ng32-CZeYrCxlYevcjyZIOgac 

Downloading...
From: https://drive.google.com/uc?id=1eMndxf7ng32-CZeYrCxlYevcjyZIOgac
To: /content/BATS_3.0.zip
100% 64.3k/64.3k [00:00<00:00, 56.1MB/s]


In [2]:
!gdown --id 1ainkSl2ThLjNq9W4W9orH3CGdZwew-zK

Downloading...
From: https://drive.google.com/uc?id=1ainkSl2ThLjNq9W4W9orH3CGdZwew-zK
To: /content/brown_raw_W.npy
75.1MB [00:03, 22.6MB/s]


In [3]:
!gdown --id  1VbC3crotxedMd5u3ZtkHoUl3k-xQ51Ht

Downloading...
From: https://drive.google.com/uc?id=1VbC3crotxedMd5u3ZtkHoUl3k-xQ51Ht
To: /content/gutenberg_raw_embeddings_dataset.npy
73.7MB [00:00, 143MB/s]


In [4]:
import nltk
import re
import numpy as np
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

In [5]:
!pip install vecto
clear_output()

In [6]:
! unzip BATS_3.0.zip
clear_output()

In [7]:
import vecto

## Choosing Data
Data for corpus :Project Gutenberg Selections and Brown Dataset from http://www.nltk.org/nltk_data/

Preprocessing inspired by :
* https://shravan-kuchkula.github.io/scrape_clean_normalize_gutenberg_text/#part-1-scrape-the-gutenberg-website : We have to lowercase everything and remove redundant characters (non-letter onces except the hypen symbol as it may be a part of word : co-operation).
Because the datasets are formed from published books or articles, I assume that there is no need in spell-checking. Also I do not change the word forms to infinitives because I want to tests their derivational and inflectional morphologies further (based on BATS datasets). Words not characters are assumed to be tokens. In fact, this is not the best choice from morfological perspective (because it results in 'cat' and 'cats' to be to non-connected words), characters as minimal unit may help to solve it but per-character processing is longer and more difficult to implement, hence, I have decided to stop at the word-level.

In [8]:
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [9]:
nltk.download('gutenberg')
nltk.download('punkt')
text_names = nltk.corpus.gutenberg.fileids()

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
def preprocess_text(word):
  # transforms everything to lower case, 
  # removes all puctuation and numbers except of dash symbol (non-invasive is one word not 2)
  # removes redundant spaces
  word = word.lower()
  part1 = re.sub(r'[^\w\s-]',"", word)
  part2 = re.sub(r'[\d+]',"", part1)
  part3 = re.sub(r' +', " ", part2)
  part4 = re.sub(r'_', "", part3)
  part5 = re.sub(r' - ', "-", part4)
  return  re.sub(r' -- ', " ", part5).strip()

In [11]:
def one_text_process(unique_words, sentences):
  sentences = [preprocess_text(" ".join(list_of_words)) for 
                    list_of_words in sentences]

  unique_words_from_text =  list(set(" ".join(sentences).split(' ')))
  unique_words_from_text = filter(
      lambda x: set(x) != set('-') and set(x) != set(''), unique_words_from_text)
  
  for w in unique_words_from_text :
    unique_words.add(w)

  return unique_words

def only_unique_words(mode='brown'): 
  unique_words = set()
  vocab_size = 0
  enumerated_words = {}
  if mode == 'gutenberg':
    text_names = nltk.corpus.gutenberg.fileids()
    for i in text_names:
      sentences = nltk.corpus.gutenberg.sents(i)
      unique_words = one_text_process(unique_words, sentences)
 
  else:
    sentences = brown.sents()
    unique_words = one_text_process(unique_words, sentences)

  unique_words = sorted(unique_words)

  for i, w in enumerate(unique_words):
    vocab_size += 1
    enumerated_words[w] = i
  return vocab_size, enumerated_words

In [12]:
# we have ~40 000 unique words in our corpus
# and ~ 2 154 000 words in total
# put mode='gutenberg' to change the dataset
vocab_size, enumerated_words = list(only_unique_words())
# vocab_size, enumerated_words = list(only_unique_words('gutenberg'))

In [13]:
!mkdir vocab

In [14]:
with open('vocab/dataset.vocab', 'w') as f:
  f.write('\n'.join(sorted(enumerated_words.keys())))

In [15]:
W = np.load('brown_raw_W.npy')
# comment the previous and uncomment currrent line to change the dataset
# W = np.load('gutenberg_raw_embeddings_dataset.npy')

# normalize embeddings
for i, row in enumerate(W):
    W[i, :] /= np.linalg.norm(row)
    
# Remove context word vectors
W = W[:len(enumerated_words), :]

np.save('vocab/embeddings_dataset_normalized.npy', W)

## Creating a Co-Occurence matrix
Now, when we have a set of unique words, we can preprocess initial texts and create a co-ocurence matrix. We will use the window size of 10 and find how often each pair of words appear in one sentence inside the window size. 

Actually, colab RAM is limited, hence, it was not possible to make dataFrame with size (number_of_unique_words, number_of_unique_words) or a dictionary with all possible combinations of the unique words. scipy.sparse. lil_matrix has worked in terms of memory but it takes really long to work, due to linear time to access. Hence, I have desided to add turple of words to a dictionary as a key if and only if the were found together. There may be the case that a same word would appear twice inside the context window, such cases are also saved. The order of words in pair is neglected (It does not matter either is was 'a' before 'b' or vice versa inside the context window. It will be added for both ('a', 'b') key and ('b', 'q') key. This is done just to make a context window in the same fashion as a moving one, just going forward, but for a particular word token both before and after it should be counted as ones inside the context window. If I have used a co-ocurance matrix in its normal form - it would be symmetric.)

In [16]:
def one_text_matrix(coocur_dict, sentences, enumerated_words, window_size = 10):
  sentences = [preprocess_text(" ".join(list_of_words)) for list_of_words in sentences]
  for sents in tqdm(sentences):
    sents_split = sents.split()
    sents_split = list(filter(
      lambda x: set(x) != set('-') and set(x) != set(''), sents_split))
    l = len(sents_split)
    for i, word1 in enumerate(sents_split):
      subsents = sents_split[i+1 : max(i + window_size, l)]
      word1 = enumerated_words[word1]
      for j, word2 in enumerate(subsents):
        word2 = enumerated_words[word2]
        name1 = (word2, word1)
        name2 = (word1, word2)
        if name1 in coocur_dict.keys():
          coocur_dict[name1] += 1 / (j + 1)
          coocur_dict[name2] += 1 / (j + 1)
        else:
          coocur_dict[name1] = 1 / (j + 1)
          coocur_dict[name2] = 1 / (j + 1)

  return coocur_dict

def build_cooucurence(enumerated_words, mode='brown', window_size = 10):
  coocur_dict = {}
  if mode == 'brown':
    sentences = brown.sents()
    coocur_dict = one_text_matrix(coocur_dict, sentences, 
                                  enumerated_words, window_size = 10)
    
  else:  
    text_names = nltk.corpus.gutenberg.fileids()
    for text in tqdm(text_names):
      sentences = nltk.corpus.gutenberg.sents(text)
      coocur_dict = one_text_matrix(coocur_dict, sentences, 
                                  enumerated_words, window_size = 10)
          
  average_coocur_score = sum(coocur_dict.values())/len(coocur_dict.values())
  print(f'Average score : {average_coocur_score}')
  # for i in coocur_dict.keys():
  # saving indexes, not words, allows to save RAM
  coocur_dict = [[i[0], i[1], coocur_dict[i]] for i in sorted(coocur_dict.keys())]

  return coocur_dict

In [17]:
# there are ~12 974 000 pairs of words - gutenberg
#           ~ 7 174 000                - brown
# please wait, this cell would take around 90 seconds
# put mode='gutenberg' to change the dataset
coocurs = build_cooucurence(enumerated_words)
# coocurs = build_cooucurence(enumerated_words, mode ='gutenberg')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



Average score : 0.7536340860230532


In [18]:
# TO SAVE MEMORY
enumerated_words = None

In [19]:
def train_glove(cooccurrences, vocab_size, vector_size = 100, 
                iterations = 50, learning_rate = 0.05, x_max = 100, alpha = 0.75):
  '''
  cooccurrences is dict (word1, word2)->score
  vector_size is the size of an output word-embedding vector
  Acknoledgement : 
  those lines are inspired by http://www.foldl.me/2014/glove-python/#implementation 
  '''

  border = 0.5 / float(vector_size + 1)
  # vector i is for word as a main one, 
  # vector i+vocab_size for the same word but in role of context word
  W = np.random.uniform( (-1)*border, border, size = (vocab_size * 2, vector_size)) 
  biases = np.random.uniform( (-1)*border, border, size = (vocab_size * 2, 1))
  gradient_squared = np.ones((vocab_size * 2, vector_size), dtype=np.float64)
  gradient_squared_biases = np.ones(vocab_size * 2, dtype=np.float64)
  data = []

  for  i_main, i_context, cooccurrence in tqdm(cooccurrences):
    # using pure indexes makes it a minute longer per iteration to run 
    # (which as ~ an additional hour for 50 iterations)
    data.append((W[i_main], W[i_context + vocab_size],
            biases[i_main],
            biases[i_context + vocab_size],
            gradient_squared[i_main], 
            gradient_squared[i_context + vocab_size],
            gradient_squared_biases[i_main],
            gradient_squared_biases[i_context + vocab_size],
            cooccurrence))

  # clering to save some space

  costs = []
  for i in tqdm(range(iterations)):
    global_cost = 0
    cost = 0
    # this is inplace operation
    np.random.shuffle(data)

    for i in tqdm(range(len(data))):

      v_main, v_context, b_main, b_context, gradsq_W_main, gradsq_W_context, gradsq_b, gradsq_b_c, cooccurrence = data[i]

      # formula (9) from the article
      weight = 0
      if cooccurrence < x_max:
        # ** is better than np.power
        # because for np.power overflow occur earlier
        weight = (cooccurrence / x_max) ** alpha
      else:
        weight = 1

      # inside the brackets - left part of formula 8
      cost_inner = (v_main @ v_context 
                      + b_main + b_context - np.log(cooccurrence))
      
      # formula 8 finished - one item inside summation
      cost = weight * (cost_inner ** 2)

      # variable to get the result for cost(formula 8)
      global_cost += cost

      # AdaGrad starts from here
      # during taking gradients we assume Note that since f(Xij) is a constant 
      # as it depended only on dataset
      # other details about diffirentiating in the text above

      # AdaGrad is 
      # x_(t+1, i) = x_(t,i)-mu*g_(t,i)/()

      scale = weight * cost_inner
      lr_scale = learning_rate * scale

      v_main -= lr_scale * v_context / (gradsq_W_main ** (1 / 2))
      v_context -= lr_scale * v_main / (gradsq_W_context ** (1 / 2))

      b_main -= lr_scale / (gradsq_b ** (1 / 2))
      b_context -= lr_scale / (gradsq_b_c ** (1 / 2))

      # sum grads for AdaGrads
      gradsq_W_main += (scale * v_context) ** 2
      gradsq_W_context += (scale * v_main) ** 2
      gradsq_b += scale ** 2
      gradsq_b_c += scale ** 2

    print(global_cost)  
    costs.append(global_cost) 

  return W, costs

In [20]:
# here it is just one itteration to show that the pipeline works
# but you can change the number of iterations if you want to wait and test everything)
vector_size = 100
W, costs = train_glove(coocurs, vocab_size, 
                       vector_size = vector_size,iterations=1)
clear_output()

np.save('raw_W.npy', W)

# W_avg = np.zeros((vocab_size, vector_size), dtype=np.float64)
# # taking avearge between context and main word embeddings
# for i in range(vocab_size):
#   W_avg[i] = (W[i] + W[i + vocab_size]) / 2

# np.save('dataset_avg.npy', W_avg)

In [ ]:
print(costs)

## BATS EVALUATON

Evaluation adopted from the official tutorial
* Please, note that documentation is not correct and vecto.benchmarks.analogy.analogy.run(model, options) does not work (https://github.com/vecto-ai/vecto/issues/73) 

* Also nornalize flag is not available from command line interface and it seems that in analogy method no normalization is required according to documentation, however, output assumes that the input embeddings are normalized. I have tried to analyze output for both normalized and non-normalized input embeddings

* The evaluation is also very long to run (around 5 hours). I understand that it is not fun at all, hence, here is the link google colab with output files : https://drive.google.com/drive/folders/1xCh1S6IeQ24RsDITs9AgKSVqUIx8w35m?usp=sharing 

* Gutenberg scores (correct out of 24500)
  * Inflectional_morphology - 21 (performed best on one-many nouns and simple past forms of vorbs (-ed))
  * Derivational_morphology - 0
  * Encyclopedic_semantics - 21 (performed best on questions about colors, I guess that is because fiction implies a lot of description also about colors)
  * Lexicographic_semantics - 26 (performed best on questions about animal group names (flock/herd) and materials (steel/wood))

* Brown non-normalized scores 
  * Inflectional_morphology - 15
  * Derivational_morphology - 5
  * Encyclopedic_semantics - 6
  * Lexicographic_semantics - 57

* Brown normalized scores 
  * Inflectional_morphology - 23 (performed best on vorbs present and past forms (-s, -ed, spend-spent))
  * Derivational_morphology - 2
  * Encyclopedic_semantics - 11 (questions about animals anologies and colors but color performance is worse compared to Gutenberg)
  * Lexicographic_semantics - 60


## Questions from the Assigment 

* **Think about how well the embedding implementation performed, e.g. did you have enough data?** 
Honestly, my embeddings seemed to perfom very poor on both datasets - brown and gutenberg ones. It is not surprising that using big context window resulted in poor catching of grammar features (derivational and inflectional morphologies). The results of sematic tasks is better but stil is far from being good. Experimenting with 2 datasets I can say that the gutenberg is much bigger and it performed better on average, hence, adding more data could help. (Also I belive a longer training could help but I did not managed to do it because I had a lot of sessions crashed.) But colab resourses would not allow it and also we should not use fiction text because it is more dificult to get semantics features out of those due to articstic forms and phrases. 
* **What happened with out-of-vocabulary words?**
Mostly, they have failed in finding the pair. But due to big number of fails even with known words I would not say that this is a good conclusion.

* **Which lexical semantic relations were picked up well, and which were not?**
You can see the scores above, for different datasets it worked differently. Normalization of vector helps but I belive that it is the impact of testing pipeline assumpttions. However, it seems that on both datasets inflectional and lexicographic performs the best

It also interesting that the answers for analogy tasks are extremely unstable with respect to the metric used, for instanse, LRCos works much faster (around an hour compared to 5 hours) but the outputs are much worse.

In [ ]:
!mkdir Inflectional_morphology_results
!mkdir Derivational_morphology_results
!mkdir Encyclopedic_semantics_results
!mkdir Lexicographic_semantics_results

In [ ]:
!python -m vecto benchmark analogy vocab/ BATS_3.0/1_Inflectional_morphology/ --path_out Inflectional_morphology_results/ --method 3CosAdd 

analogy ['vocab_guten/', 'BATS_3.0/1_Inflectional_morphology/', '--path_out', 'Inflectional_morphology_results_guten/', '--method', '3CosAdd']
running  analogy
SHAPE: (46909, 100)
vocab size: 46909
100% (50 of 50) |#########################| Elapsed Time: 1:01:58 Time:  1:01:58
100% (50 of 50) |#########################| Elapsed Time: 0:55:34 Time:  0:55:34
100% (50 of 50) |#########################| Elapsed Time: 0:49:33 Time:  0:49:33
100% (50 of 50) |#########################| Elapsed Time: 0:41:56 Time:  0:41:56
100% (50 of 50) |#########################| Elapsed Time: 0:34:36 Time:  0:34:36
100% (50 of 50) |#########################| Elapsed Time: 0:28:59 Time:  0:28:59
100% (50 of 50) |#########################| Elapsed Time: 0:23:39 Time:  0:23:39
100% (50 of 50) |#########################| Elapsed Time: 0:17:23 Time:  0:17:23
100% (50 of 50) |#########################| Elapsed Time: 0:11:51 Time:  0:11:51
100% (50 of 50) |#########################| Elapsed Time: 0:05:57 Time:  

In [ ]:
!python -m vecto benchmark analogy vocab/ BATS_3.0/2_Derivational_morphology/ --path_out Derivational_morphology_results/ --method 3CosAdd 

analogy ['vocab_guten/', 'BATS_3.0/2_Derivational_morphology/', '--path_out', 'Derivational_morphology_results_guten/', '--method', '3CosAdd']
running  analogy
SHAPE: (46909, 100)
vocab size: 46909
100% (50 of 50) |#########################| Elapsed Time: 1:08:24 Time:  1:08:24
100% (50 of 50) |#########################| Elapsed Time: 1:01:25 Time:  1:01:25
100% (50 of 50) |#########################| Elapsed Time: 0:55:26 Time:  0:55:26
100% (50 of 50) |#########################| Elapsed Time: 0:49:43 Time:  0:49:43
100% (50 of 50) |#########################| Elapsed Time: 0:42:27 Time:  0:42:27
100% (50 of 50) |#########################| Elapsed Time: 0:35:02 Time:  0:35:02
100% (50 of 50) |#########################| Elapsed Time: 0:27:15 Time:  0:27:15
100% (50 of 50) |#########################| Elapsed Time: 0:19:50 Time:  0:19:50
100% (50 of 50) |#########################| Elapsed Time: 0:13:19 Time:  0:13:19
100% (50 of 50) |#########################| Elapsed Time: 0:06:34 Time:  

In [ ]:
!python -m vecto benchmark analogy vocab/ BATS_3.0/3_Encyclopedic_semantics/ --path_out Encyclopedic_semantics_results/ --method 3CosAdd

analogy ['vocab_guten/', 'BATS_3.0/3_Encyclopedic_semantics/', '--path_out', 'Encyclopedic_semantics_results_guten/', '--method', '3CosAdd']
running  analogy
SHAPE: (46909, 100)
vocab size: 46909
 36% (18 of 50) |#########                | Elapsed Time: 0:02:46 ETA:   0:05:17

In [ ]:
!python -m vecto benchmark analogy vocab/ BATS_3.0/4_Lexicographic_semantics/ --path_out Lexicographic_semantics_results/ --method 3CosAdd